In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time
import warnings

# Suppress warnings for a cleaner output
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Determine the device to use (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}\n")

# ============================================================================
# MODIFIED SIMPLE CNN ARCHITECTURE (FC1 and FC2 Layers Removed)
# ============================================================================
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        # Directly connect the output of conv3 (after pooling) to the final fc3 layer
        # The output size of conv3 after 3 pooling layers (each reducing by factor of 2)
        # from a 32x32 input is 32 / (2*2*2) = 4x4.
        # So, the input features to fc3 will be 128 channels * 4 * 4.
        self.fc3 = nn.Linear(128 * 4 * 4, num_classes)
        
        # Removed self.fc1, self.fc2, and self.dropout

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        
        # Flatten the output for the FC layer
        x = x.view(x.size(0), -1) 
        
        # Directly pass to the final classification layer
        return self.fc3(x)

# ============================================================================
# DATA LOADING (CIFAR-10)
# ============================================================================
def load_cifar10_data(batch_size=128):
    """
    Loads and preprocesses CIFAR-10 dataset.
    """
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010))
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010))
    ])
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)
    return trainloader, testloader

# ============================================================================
# TRAINING AND EVALUATION FUNCTIONS
# ============================================================================
def train_model(model, trainloader, testloader, epochs=10, lr=0.001, verbose=True):
    """
    Trains the given PyTorch model.
    """
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        correct, total, loss_sum = 0, 0, 0.0
        for inputs, targets in trainloader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)
            loss_sum += loss.item()
        if verbose:
            print(f"Epoch {epoch+1}/{epochs}: Loss={loss_sum / len(trainloader):.4f}, Accuracy={100.*correct/total:.2f}%")
    return model

def evaluate_model(model, dataloader):
    """
    Evaluates the given PyTorch model.
    """
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)
    return 100. * correct / total

# ============================================================================
# PARAMETER COUNTING FUNCTIONS
# ============================================================================
def count_model_parameters(model):
    """Count total trainable parameters in a model"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# ============================================================================
# MAIN SCRIPT EXECUTION (Modified)
# ============================================================================

def run_simplified_cnn_experiment():
    """
    Runs the experiment with FC1 and FC2 layers removed.
    """
    print("Loading CIFAR-10 data...")
    trainloader, testloader = load_cifar10_data()

    # --- Step 1: Initialize and train the modified model ---
    print("\n--- Training Modified SimpleCNN (FC1 & FC2 removed) ---")
    
    # Create an instance of the modified SimpleCNN
    modified_model = SimpleCNN()
    
    # Train the modified model for 5 epochs, same as the original baseline
    train_model(modified_model, trainloader, testloader, epochs=5, lr=0.001, verbose=True)
    
    # --- Step 2: Evaluate the modified model ---
    final_accuracy = evaluate_model(modified_model, testloader)
    total_params = count_model_parameters(modified_model)

    print(f"\nFinal Accuracy of Modified Model (FC1 & FC2 removed): {final_accuracy:.2f}%")
    print(f"Total Parameters of Modified Model: {total_params:,}")

if __name__ == "__main__":
    run_simplified_cnn_experiment()

PyTorch version: 2.5.1
Device: cuda

Loading CIFAR-10 data...
Files already downloaded and verified
Files already downloaded and verified

--- Training Modified SimpleCNN (FC1 & FC2 removed) ---
Epoch 1/5: Loss=1.5779, Accuracy=42.67%
Epoch 2/5: Loss=1.1963, Accuracy=57.31%
Epoch 3/5: Loss=1.0274, Accuracy=64.22%
Epoch 4/5: Loss=0.9307, Accuracy=67.72%
Epoch 5/5: Loss=0.8613, Accuracy=69.98%

Final Accuracy of Modified Model (FC1 & FC2 removed): 71.86%
Total Parameters of Modified Model: 113,738
